In [ ]:
from elasticsearch import Elasticsearch
import os
import glob
import PyPDF2
import PyPDF3
import pandas as pd

In [ ]:
data= pd.read_excel("NGC_EC_DA_P1.xlsx",sheet_name=0)

In [ ]:
for i in range(len(data['Cd_AR'])):
    check_value= data['mpn_validated'][i]
    string=str(data['Cd_AR'][i])
    
    if (check_value=='NO VALUE'):
        try:
            aftr_split=string.split('-',maxsplit=1)
            if len(aftr_split)<2:
                F_str=aftr_split[0]
            else:
                F_str=aftr_split[1]
            mpn_extracted= mpn_extracted.append({'Original':string,'Extracted':F_str},ignore_index=True)
        except (IndexError,AttributeError):
            print(string)
    else:
        mpn_extracted= mpn_extracted.append({'Original':string,'Extracted':check_value},ignore_index=True)
        
    

In [ ]:
mpn_extracted.to_excel('mpn_extracted_values_2.xlsx')

In [ ]:
search_str=mpn_extracted['Extracted']

In [ ]:
for j in range(len(fix_part_num['Original'])):
    finder=fix_part_num['Original'][j]
    ind=(mpn_extracted[mpn_extracted['Original']==finder]).index[0]
    mpn_extracted['Extracted'][ind]=fix_part_num['Extracted'][j]
    

In [ ]:
fix_part_num= pd.read_excel("mpn_extracted_values_1.xlsx",sheet_name=1)

In [ ]:
for k in range(len(fix_part_num['Extracted'])):
    change=fix_part_num['Extracted'][k]
    assign= (change.split('-',maxsplit=1))[1]
    fix_part_num['Extracted'][k]=assign

In [ ]:
search_str=fix_part_num['Extracted']

In [ ]:
os.chdir("C:/Users/Ganesh P Umarani/Desktop/all_pdfs")
files=glob.glob("*.*")

#Function to read pdf using PyPDF2

In [ ]:
def extractpdffiles(files):
    this_loc=1
    df=pd.DataFrame(columns=("name","content"))
    
    for file in [files]:
        try:
            pdfFileObj=open(file,'rb')
            #pdfReader=PyPDF2.PdfFileReader(pdfFileObj)
            pdfReader=PyPDF3.PdfFileReader(pdfFileObj)
            print(file)
            if pdfReader.isEncrypted:
                try:
                    pdfReader.decrypt('')
                except Exception as e :
                    print('{} --- > {}'.format(file, e))
                    continue
                
            n_pages=pdfReader.numPages
            this_doc=''
            for i in range(n_pages):
                pageObj=pdfReader.getPage(i)
                this_text=pageObj.extractText()
                this_doc+=this_text
            df.loc[this_loc]= file , this_doc
            this_loc = this_loc +1
        except Exception as e:
            print('{} --- > {}'.format(file, e))
    return df

In [ ]:
df = extractpdffiles(files)

In [ ]:
df=df[df.content != '']

In [ ]:
df.reset_index(drop=True, inplace=True)

# Create Index

In [ ]:
es = Elasticsearch()

In [ ]:
es.indices.get_alias("*")

In [ ]:
col_names=df.columns
for row_number in range(df.shape[0]):
    body=dict([(name,str(df.iloc[row_number][name])) for name in col_names])
    es.index(index='all_2',doc_type='catalogue',body=body)

In [ ]:
result = pd.DataFrame(columns=['part_num','file_names'])
for searchstring in ['496H220']:
    search_results=es.search(index='all_2',
                        body={"_source":"name",
                             'query':{
                                 'match_phrase':{"content": searchstring},
                                    }
                             })
    
    res_list=search_results['hits']['hits']
    for i in range(len(res_list)):
        res_dict=res_list[i]
        result=result.append({'part_num':searchstring,'file_names':res_dict['_source']['name']},ignore_index=True)
    print("search string: {}  result list: {}".format(searchstring,len(res_list)))
        
        

In [ ]:
result.to_excel("ES_output_3.xlsx")

In [ ]:
joined_part_num= mpn_extracted.merge(result,on='Extracted',how='outer')

In [ ]:
Joined_all_fields.rename({'Original':'Cd_AR'},axis=1,inplace=True)

In [ ]:
joined_part_num.to_excel('buffer_check.xlsx')

In [ ]:
Joined_all_fields.columns

In [ ]:
Joined_all_fields.columns

In [ ]:
to_drop=['Row ID', 'Cd_AR clean num', 'Owner', 'Obsoleto',
       'Cd_ARStato', 'TimeIns', 'dt_lastmov', 'ProdClass',
       'Family', 'Group', 'Status', 'MinimumLot', 'ReorderLot', 'VAT',
       'Weight', 'Repertorio', 'cnd', 'Cd_ARMisura', 'ResponsabileLinea', 'Brand', 'SUPPLIER',
       'SUPPLIER NAME', 'manufacturer_validated',
       'corporate_parent_validated', 'supplier_validated', 'brand_validated',
       'weblink_product_page', 'weblink_mnf', 'weblink_corporate_parent',
       'comment', 'Additional Info', 'value_info']

In [ ]:
Joined_all_fields.drop(to_drop,axis=1,inplace=True)

In [ ]:
data['Cd_AR']=data['Cd_AR'].astype(str,copy=False)

In [ ]:
Joined_all_fields=pd.read_excel('buffer_check.xlsx')

In [ ]:
Joined_all_fields=data.merge(Joined_all_fields,on='Cd_AR',how='outer')

In [ ]:
type(data['Cd_AR'][131413])

In [ ]:
Joined_all_fields.head()

In [ ]:
len(Joined_all_fields)

In [ ]:
Joined_all_fields[Joined_all_fields['mpn_validated']=='7145']

In [ ]:
Joined_all_fields.to_excel('Output_with_all Fields_6.xlsx')

In [ ]:
pwd